# Customer Segmentation

In [1]:

import snowflake.connector
import pandas as pd

Importing the data from Snowflake to a dataframe.

Masking the credentials

In [2]:
ctx = snowflake.connector.connect(
user='*******',
password='*******',
account='*******'
)
query = "SELECT PR.PRODUCT_ID,O.ORDER_ID,O.CUSTOMER_ID, PR.PRODUCT_CATEGORY_NAME, OP.PAYMENT_VALUE,O.ORDER_DELIVERED_CUSTOMER_DATE,OC.CUSTOMER_UNIQUE_ID,ORD.REVIEW_SCORE, OC.CUSTOMER_CITY FROM OLIST.PUBLIC.OLIST_PRODUCTS_DATASET AS PR JOIN OLIST.PUBLIC.OLIST_ORDER_ITEMS_DATASET AS OI ON PR.PRODUCT_ID = OI.PRODUCT_ID JOIN OLIST.PUBLIC.OLIST_ORDERS_DATASET O ON OI.ORDER_ID = O.ORDER_ID JOIN OLIST.PUBLIC.OLIST_CUSTOMERS_DATASET OC ON O.CUSTOMER_ID = OC.CUSTOMER_ID JOIN OLIST.PUBLIC.OLIST_ORDER_PAYMENTS_DATASET OP ON O.ORDER_ID = OP.ORDER_ID JOIN OLIST.PUBLIC.OLIST_ORDER_REVIEWS_DATASET ORD ON O.ORDER_ID = ORD.ORDER_ID WHERE O.ORDER_STATUS = 'delivered'"
data = pd.read_sql(query, ctx)
data

,PRODUCT_ID,ORDER_ID,CUSTOMER_ID,PRODUCT_CATEGORY_NAME,PAYMENT_VALUE,ORDER_DELIVERED_CUSTOMER_DATE,CUSTOMER_UNIQUE_ID,REVIEW_SCORE,CUSTOMER_CITY
0,1e9e8ef04dbcff4541ed26657ea517e5,e17e4f88e31525f7deef66779844ddce,f8a3e963a310aa58b60a5b1fed5bceb5,perfumaria,18.30,2018-04-27 16:42:17,b1a1199364a4a7fe27c4486ab63f550d,5,mogi-guacu
1,3aa071139cb16b67ca9e5dea641aaa2f,5236307716393b7114b53ee991f36956,03fc97548af8f58fefc768d12b546c9c,artes,265.99,2018-02-16 13:27:08,4b86049cb99e4aa774031daa9cd18f18,5,rio de janeiro
2,96bd76ec8810374ed1b65e291975717f,01f66e58769f84129811d43eefd187fb,e41819d1c95c12c9ce495b630eab8aee,esporte_lazer,87.62,2018-07-11 23:24:16,f63805d9c7edb84d92413af34b86a39c,5,sao paulo
3,cef67bcfe19066a932b7673e239eb23d,143d00a4f2dde4e0364ee1821577adb3,322162b5ca010c2b059cb5224dd818b1,bebes,121.84,2018-08-06 15:14:32,619e926d09b26efbd5180368b1ddc874,1,sao paulo
4,9dc1a7de274444849c219cff195d0b71,86cafb8794cb99a9b1b77fc8e48fbbbb,c11c31965ff02cc1d7132df8edfcbc22,utilidades_domesticas,46.19,2018-04-13 22:41:44,ad353b4fb0e294adc4eda48af73e68a6,5,sao paulo
...,...,...,...,...,...,...,...,...,...
114854,83808703fc0706a22e264b9d75f04a2e,59e88b7d3675e89aceaf86f372d3bc9a,e750210dde7e68dbd5194c0809e97f7e,informatica_acessorios,92.02,2018-01-15 20:39:35,4d1bbadcfd9bdc738e54599627870319,4,campo florido
114855,83808703fc0706a22e264b9d75f04a2e,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,informatica_acessorios,159.54,2018-03-23 20:08:40,2cc893f1618b7317170ba2456cf76117,5,paramirim
114856,83808703fc0706a22e264b9d75f04a2e,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,informatica_acessorios,159.54,2018-03-23 20:08:40,2cc893f1618b7317170ba2456cf76117,5,paramirim
114857,83808703fc0706a22e264b9d75f04a2e,6e4465d771f02e4fe335225de3c6c043,0f139d1b208a62a37671423e5eff5016,informatica_acessorios,159.54,2018-03-23 20:08:40,2cc893f1618b7317170ba2456cf76117,5,paramirim


In [3]:
data['ORDER_DELIVERED_CUSTOMER_DATE'] = data.ORDER_DELIVERED_CUSTOMER_DATE.apply(lambda x: pd.to_datetime(x, format="%Y-%m-%d"))

In [4]:
data = data.drop_duplicates()
data.count()

PRODUCT_ID                       103572
ORDER_ID                         103572
CUSTOMER_ID                      103572
PRODUCT_CATEGORY_NAME            102117
PAYMENT_VALUE                    103572
ORDER_DELIVERED_CUSTOMER_DATE    103564
CUSTOMER_UNIQUE_ID               103572
REVIEW_SCORE                     103572
CUSTOMER_CITY                    103572
dtype: int64

In [5]:
data.isnull().sum()

PRODUCT_ID                          0
ORDER_ID                            0
CUSTOMER_ID                         0
PRODUCT_CATEGORY_NAME            1455
PAYMENT_VALUE                       0
ORDER_DELIVERED_CUSTOMER_DATE       8
CUSTOMER_UNIQUE_ID                  0
REVIEW_SCORE                        0
CUSTOMER_CITY                       0
dtype: int64

Recency

In [6]:
cust_max_purchase = data.groupby('CUSTOMER_ID').ORDER_DELIVERED_CUSTOMER_DATE.max().reset_index()
cust_max_purchase.columns = ['CUSTOMER_ID','MAX_PURCHASE_DATE']
cust_max_purchase

,CUSTOMER_ID,MAX_PURCHASE_DATE
0,00012a2ce6f8dcda20d059ce98491703,2017-11-28 15:41:30
1,000161a058600d5901f007fab4c27140,2017-07-25 18:57:33
2,0001fd6190edaaf884bcaf3d49edf079,2017-03-06 08:57:49
3,0002414f95344307404f0ace7a26f1d5,2017-09-13 20:06:02
4,000379cdec625522490c315e70c7a9fb,2018-04-13 20:21:08
...,...,...
95826,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-21 19:24:46
95827,fffecc9f79fd8c764f843e9951b11341,2018-04-10 17:20:49
95828,fffeda5b6d849fbd39689bb92087f431,2018-06-08 18:03:31
95829,ffff42319e9b2d713724ae527742af25,2018-06-18 18:33:05


Recency Calculation

In [7]:
cust_max_purchase['RECENCY'] = (cust_max_purchase['MAX_PURCHASE_DATE'].max() - cust_max_purchase['MAX_PURCHASE_DATE']).dt.days
cust_max_purchase

,CUSTOMER_ID,MAX_PURCHASE_DATE,RECENCY
0,00012a2ce6f8dcda20d059ce98491703,2017-11-28 15:41:30,322.0
1,000161a058600d5901f007fab4c27140,2017-07-25 18:57:33,448.0
2,0001fd6190edaaf884bcaf3d49edf079,2017-03-06 08:57:49,590.0
3,0002414f95344307404f0ace7a26f1d5,2017-09-13 20:06:02,398.0
4,000379cdec625522490c315e70c7a9fb,2018-04-13 20:21:08,186.0
...,...,...,...
95826,fffcb937e9dd47a13f05ecb8290f4d3e,2018-03-21 19:24:46,209.0
95827,fffecc9f79fd8c764f843e9951b11341,2018-04-10 17:20:49,189.0
95828,fffeda5b6d849fbd39689bb92087f431,2018-06-08 18:03:31,130.0
95829,ffff42319e9b2d713724ae527742af25,2018-06-18 18:33:05,120.0


In [8]:
data_user = cust_max_purchase.drop(['MAX_PURCHASE_DATE'],axis=1)

In [9]:
data_user.RECENCY.describe()

count    95823.000000
mean       275.241111
std        152.275819
min          0.000000
25%        154.000000
50%        256.000000
75%        386.000000
max        735.000000
Name: RECENCY, dtype: float64

In [10]:
data_user.head()

,CUSTOMER_ID,RECENCY
0,00012a2ce6f8dcda20d059ce98491703,322.0
1,000161a058600d5901f007fab4c27140,448.0
2,0001fd6190edaaf884bcaf3d49edf079,590.0
3,0002414f95344307404f0ace7a26f1d5,398.0
4,000379cdec625522490c315e70c7a9fb,186.0


Frequency Calculation

In [11]:
frequency_df = data.groupby(by=['CUSTOMER_ID'], as_index=False)['ORDER_DELIVERED_CUSTOMER_DATE'].count()
frequency_df.columns = ['CUSTOMER_ID','FREQUENCY']
frequency_df.head()

,CUSTOMER_ID,FREQUENCY
0,00012a2ce6f8dcda20d059ce98491703,1
1,000161a058600d5901f007fab4c27140,1
2,0001fd6190edaaf884bcaf3d49edf079,1
3,0002414f95344307404f0ace7a26f1d5,1
4,000379cdec625522490c315e70c7a9fb,1


Monetary value Calculation

In [12]:
monetary_df = data.groupby(by='CUSTOMER_ID',as_index=False).agg({'PAYMENT_VALUE': 'sum'})
monetary_df.columns = ['CUSTOMER_ID','MONETARY']
monetary_df.head()

,CUSTOMER_ID,MONETARY
0,00012a2ce6f8dcda20d059ce98491703,114.74
1,000161a058600d5901f007fab4c27140,67.41
2,0001fd6190edaaf884bcaf3d49edf079,195.42
3,0002414f95344307404f0ace7a26f1d5,179.35
4,000379cdec625522490c315e70c7a9fb,107.01


Merging the dataframes to build a RMF table

In [13]:
#merge recency dataframe with frequency dataframe
temp_df = data_user.merge(frequency_df,on='CUSTOMER_ID')
temp_df.head()

,CUSTOMER_ID,RECENCY,FREQUENCY
0,00012a2ce6f8dcda20d059ce98491703,322.0,1
1,000161a058600d5901f007fab4c27140,448.0,1
2,0001fd6190edaaf884bcaf3d49edf079,590.0,1
3,0002414f95344307404f0ace7a26f1d5,398.0,1
4,000379cdec625522490c315e70c7a9fb,186.0,1


In [14]:
#merge with monetary dataframe to get a table with the 3 columns
rfm_df = temp_df.merge(monetary_df,on='CUSTOMER_ID')
#use CustomerID as index
rfm_df.set_index('CUSTOMER_ID',inplace=True)
#check the head
rfm_df.head()

,RECENCY,FREQUENCY,MONETARY
CUSTOMER_ID,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74
000161a058600d5901f007fab4c27140,448.0,1,67.41
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35
000379cdec625522490c315e70c7a9fb,186.0,1,107.01


In [15]:
customers_rank = rfm_df
# Create a new column that is the rank of the value of coverage in ascending order
customers_rank['RANK'] = customers_rank['MONETARY'].rank(ascending=0)

customers_rank.sort_values('RANK',ascending=True)

,RECENCY,FREQUENCY,MONETARY,RANK
CUSTOMER_ID,,,,
71901689c5f3e5adc27b1dd16b33f0b8,132.0,5,15978.65,1.0
1617b1357756262bfa56ab541c47bc16,364.0,1,13664.08,2.0
0d861a5e4dd6a9079d89e1330848f0ab,57.0,8,9258.24,3.0
cc803a2c412833101651d3f90ca7de24,133.0,2,8891.00,4.0
30bb84b541c96af98ba7d90b9ebf35d0,313.0,7,8697.99,5.0
...,...,...,...,...
da9d48cb8449edad3619186b3dd0c180,62.0,2,10.06,95827.0
a790343ca6f3fee08112d678b43aa7c5,48.0,2,9.59,95828.0
c5ea6b40204131fb48bdf66d273d7f15,426.0,2,8.86,95829.0


Defining RFM Quartiles

In [16]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

,RECENCY,FREQUENCY,MONETARY,RANK
0.25,154.0,1.0,61.99,23958.5
0.50,256.0,1.0,106.29,47921.0
0.75,386.0,1.0,181.79,71873.0


In [17]:
quantiles.to_dict()

{'RECENCY': {0.25: 154.0, 0.5: 256.0, 0.75: 386.0},
 'FREQUENCY': {0.25: 1.0, 0.5: 1.0, 0.75: 1.0},
 'MONETARY': {0.25: 61.99, 0.5: 106.29, 0.75: 181.79000000000002},
 'RANK': {0.25: 23958.5, 0.5: 47921.0, 0.75: 71873.0}}

In [18]:
# Arguments (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Arguments (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [19]:
#create rfm segmentation table
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['RECENCY'].apply(RScore, args=('RECENCY',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['FREQUENCY'].apply(FMScore, args=('FREQUENCY',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['MONETARY'].apply(FMScore, args=('MONETARY',quantiles,))

In [20]:
rfm_segmentation.head()

,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile
CUSTOMER_ID,,,,,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74,44399.5,2,1,3
000161a058600d5901f007fab4c27140,448.0,1,67.41,68024.0,1,1,2
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42,21382.0,1,1,4
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35,24371.0,1,1,3
000379cdec625522490c315e70c7a9fb,186.0,1,107.01,47513.5,3,1,3


RFM Score calculation

In [21]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile,RFMScore
CUSTOMER_ID,,,,,,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74,44399.5,2,1,3,213
000161a058600d5901f007fab4c27140,448.0,1,67.41,68024.0,1,1,2,112
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42,21382.0,1,1,4,114
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35,24371.0,1,1,3,113
000379cdec625522490c315e70c7a9fb,186.0,1,107.01,47513.5,3,1,3,313


In [22]:
print("Best Customers: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Loyal Customers: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Big Spenders: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('Almost Lost: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Lost Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Lost Cheap Customers: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Best Customers:  873
Loyal Customers:  6050
Big Spenders:  23957
Almost Lost:  893
Lost Customers:  860
Lost Cheap Customers:  5924


In [23]:
rfm_segmentation=rfm_segmentation.reset_index()

In [24]:

#Regular, Best, occasional
#rfm_segmentation.loc[rfm_segmentation['R_Quartile']==1,'Type']= 'Lost'
rfm_segmentation.loc[(rfm_segmentation['R_Quartile']>2) |(rfm_segmentation['F_Quartile']>2) | (rfm_segmentation['M_Quartile']<4),'Type']= 'Regular'
rfm_segmentation.loc[(rfm_segmentation['R_Quartile']<=2) |(rfm_segmentation['F_Quartile']<=2) | (rfm_segmentation['M_Quartile']<4),'Type']= 'Occassional'
#rfm_segmentation.loc[(rfm_segmentation['R_Quartile']<2) |(rfm_segmentation['F_Quartile']>2) | (rfm_segmentation['M_Quartile']<rfm_segmentation.loc[(rfm_segmentation['R_Quartile']<=2) |(rfm_segmentation['F_Quartile']<=2) | (rfm_segmentation['M_Quartile']==4),'Type']= 'Lost_Big_Spenders'
rfm_segmentation.loc[rfm_segmentation['RFMScore']=='444','Type']= 'Best'

In [26]:
## Adding segments to the database
#data=pd.merge(data,rfm_segmentation, how ='left', left_on='CUSTOMER_ID', right_on='CUSTOMER_ID')
data
data.to_csv("CustomerSegments.csv", index=False)

Defining customer segments

In [62]:
Best = data[data['Type']=='Best']
Occassional = data[data['Type']=='Occassional']
Regular = data[data['Type']=='Regular']

In [64]:
#reg_occasional = pd.concat([Regular,Occassional])
#best_reg_occasional =  pd.concat([regular,lost,top])
Occassional.to_csv("occassional.csv", index=False)
Best.to_csv("best.csv", index=False)
Regular.to_csv("Regular.csv", index=False)

In [69]:
print('Best:\n',Best.count())
print('\n\nRegular:\n',Regular.count())
print('\n\nOccassional:\n',Occassional.count())

Best:
 PRODUCT_ID                       1999
ORDER_ID                         1999
CUSTOMER_ID                      1999
PRODUCT_CATEGORY_NAME            1988
PAYMENT_VALUE                    1999
ORDER_DELIVERED_CUSTOMER_DATE    1999
CUSTOMER_UNIQUE_ID               1999
REVIEW_SCORE                     1999
CUSTOMER_CITY                    1999
RECENCY                          1999
FREQUENCY                        1999
MONETARY                         1999
RANK                             1999
R_Quartile                       1999
F_Quartile                       1999
M_Quartile                       1999
RFMScore                         1999
Type                             1999
dtype: int64


Regular:
 PRODUCT_ID                       1728
ORDER_ID                         1728
CUSTOMER_ID                      1728
PRODUCT_CATEGORY_NAME            1715
PAYMENT_VALUE                    1728
ORDER_DELIVERED_CUSTOMER_DATE    1728
CUSTOMER_UNIQUE_ID               1728
REVIEW_SCORE      

In [29]:
rfm_segmentation

,RECENCY,FREQUENCY,MONETARY,RANK,R_Quartile,F_Quartile,M_Quartile,RFMScore,Type
CUSTOMER_ID,,,,,,,,,
00012a2ce6f8dcda20d059ce98491703,322.0,1,114.74,44399.5,2,1,3,213,Occassional
000161a058600d5901f007fab4c27140,448.0,1,67.41,68024.0,1,1,2,112,Occassional
0001fd6190edaaf884bcaf3d49edf079,590.0,1,195.42,21382.0,1,1,4,114,Occassional
0002414f95344307404f0ace7a26f1d5,398.0,1,179.35,24371.0,1,1,3,113,Occassional
000379cdec625522490c315e70c7a9fb,186.0,1,107.01,47513.5,3,1,3,313,Occassional
...,...,...,...,...,...,...,...,...,...
fffcb937e9dd47a13f05ecb8290f4d3e,209.0,1,91.91,54482.0,3,1,2,312,Occassional
fffecc9f79fd8c764f843e9951b11341,189.0,3,81.36,59860.0,3,4,2,342,Occassional
fffeda5b6d849fbd39689bb92087f431,130.0,1,63.13,71013.0,4,1,2,412,Occassional


In [65]:
rfm_segmentation['Type'].unique()

array(['Occassional', 'Best', 'Regular'], dtype=object)